In [1]:
from benchmark import impl
from benchmark import alphaxutils
from decimal import Decimal
import pandas as pd

Order = alphaxutils.Order
tradeBySourceAmount = alphaxutils.tradeBySourceAmount
tradeByTargetAmount = alphaxutils.tradeByTargetAmount
AlphaRouter = alphaxutils.AlphaRouter
CarbonOrderUI = alphaxutils.CarbonOrderUI
assertAlmostEqual = alphaxutils.assertAlmostEqual

In [2]:
threshold_orders = 6
inputAmount = Decimal('5000')
support_partial = False

order_params = {
    'liquidity':Decimal('5000'),
    'highestRate':Decimal('2500'),
    'lowestRate':Decimal('2000'),
    'marginalRate':Decimal('2500'),
    }
order_params2 = {
    'liquidity':Decimal('4000'),
    'highestRate':Decimal('2000'),
    'lowestRate':Decimal('1666'),
    'marginalRate':Decimal('2000'),
    }

In [3]:
orders = []
for i in range(5):
    orders += [Order(order_params)]
    orders += [Order(order_params2)]

indexes = list(range(len(orders)))   

orders[0].z

Decimal('5E+3')

In [4]:
total_liquidity = sum(o.y for o in orders)
total_liquidity

Decimal('45000')

# Trade Functions
- Note they use pure math so will exceed the liquidity of an order

In [19]:
# trade 1 ETH into the first order 
tradeBySourceAmount(1, orders[0])

(1, Decimal('2374.6507262194609015632130899344576184001508194173'))

In [20]:
# trade out exactly 2374.6507262194609015632130899344576184001508194173... USDC
tradeByTargetAmount(Decimal('2374.6507262194609015632130899344576184001508194173'), orders[0])

(Decimal('0.99999999999999999999999999999999999999999999999998'),
 Decimal('2374.6507262194609015632130899344576184001508194173'))

# Match by Target
1. hypothetical_output_amt for full inputAmount

In [7]:
hypothetical_output_amts = {
            i: tradeByTargetAmount(x=inputAmount, order=orders[i])[0] for i in indexes
        }
hypothetical_output_amts

{0: Decimal('2.2360679774997896964091736687312762354406183596117'),
 1: Decimal('2.8062757546743274149851601968224106388749889761598'),
 2: Decimal('2.2360679774997896964091736687312762354406183596117'),
 3: Decimal('2.8062757546743274149851601968224106388749889761598'),
 4: Decimal('2.2360679774997896964091736687312762354406183596117'),
 5: Decimal('2.8062757546743274149851601968224106388749889761598'),
 6: Decimal('2.2360679774997896964091736687312762354406183596117'),
 7: Decimal('2.8062757546743274149851601968224106388749889761598'),
 8: Decimal('2.2360679774997896964091736687312762354406183596117'),
 9: Decimal('2.8062757546743274149851601968224106388749889761598')}

In [8]:
actual_output_amts = {
            i: tradeByTargetAmount(x=orders[i].y, order=orders[i])[0] for i in indexes
        }
actual_output_amts

{0: Decimal('2.2360679774997896964091736687312762354406183596117'),
 1: Decimal('2.1913285395639155343136038848501460223623844634990'),
 2: Decimal('2.2360679774997896964091736687312762354406183596117'),
 3: Decimal('2.1913285395639155343136038848501460223623844634990'),
 4: Decimal('2.2360679774997896964091736687312762354406183596117'),
 5: Decimal('2.1913285395639155343136038848501460223623844634990'),
 6: Decimal('2.2360679774997896964091736687312762354406183596117'),
 7: Decimal('2.1913285395639155343136038848501460223623844634990'),
 8: Decimal('2.2360679774997896964091736687312762354406183596117'),
 9: Decimal('2.1913285395639155343136038848501460223623844634990')}

In [9]:
# ordered_amts = {
#     j: hypothetical_output_amts[j]
#     for j in sorted(
#         indexes, key=lambda i: hypothetical_output_amts[i], reverse=False
#     )
# }
# ordered_amts

In [10]:
# ordered_actual_amts = {
#     j: actual_output_amts[j]
#     for j in sorted(
#         indexes, key=lambda i: actual_output_amts[i], reverse=False
#     )
# }
# ordered_actual_amts

In [11]:
ordered_associated_liquidity = {i:orders[i].y for i in actual_output_amts.keys()}
ordered_associated_liquidity

{0: Decimal('5000'),
 1: Decimal('4000'),
 2: Decimal('5000'),
 3: Decimal('4000'),
 4: Decimal('5000'),
 5: Decimal('4000'),
 6: Decimal('5000'),
 7: Decimal('4000'),
 8: Decimal('5000'),
 9: Decimal('4000')}

In [12]:
results = pd.DataFrame(
    [
        hypothetical_output_amts.keys(),
        hypothetical_output_amts.values(),
        actual_output_amts.keys(),
        actual_output_amts.values(),
    ],
    index=[
        "indexes",
        "hypothetical_output_amts",
        "indexes_b",
        "actual_output_amts",
    ],
)
results = results.T.copy()
assert(list(results.indexes) == list(results.indexes_b))

results.sort_values(
    by=["actual_output_amts", "hypothetical_output_amts", 'indexes'], ascending=[True, True, True], inplace=True
)

results2 = pd.DataFrame(
    [
        ordered_associated_liquidity.keys(),
        ordered_associated_liquidity.values(),
    ],
    index=[
        "ordered_associated_liquidity_keys",
        "ordered_associated_liquidity",
    ],
)
results2 = results2.T.copy()

results = pd.merge(results, results2, how='left', left_on = 'indexes', right_on='ordered_associated_liquidity_keys')

results.fillna(0, inplace=True)
results.reset_index(inplace=True, drop=True)
results

,indexes,hypothetical_output_amts,indexes_b,actual_output_amts,ordered_associated_liquidity_keys,ordered_associated_liquidity
0,1,2.80627575467432741498516019682241063887498897...,1,2.19132853956391553431360388485014602236238446...,1,4000
1,3,2.80627575467432741498516019682241063887498897...,3,2.19132853956391553431360388485014602236238446...,3,4000
2,5,2.80627575467432741498516019682241063887498897...,5,2.19132853956391553431360388485014602236238446...,5,4000
3,7,2.80627575467432741498516019682241063887498897...,7,2.19132853956391553431360388485014602236238446...,7,4000
4,9,2.80627575467432741498516019682241063887498897...,9,2.19132853956391553431360388485014602236238446...,9,4000
5,0,2.23606797749978969640917366873127623544061835...,0,2.23606797749978969640917366873127623544061835...,0,5000
6,2,2.23606797749978969640917366873127623544061835...,2,2.23606797749978969640917366873127623544061835...,2,5000
7,4,2.23606797749978969640917366873127623544061835...,4,2.23606797749978969640917366873127623544061835...,4,5000
8,6,2.23606797749978969640917366873127623544061835...,6,2.23606797749978969640917366873127623544061835...,6,5000
9,8,2.23606797749978969640917366873127623544061835...,8,2.23606797749978969640917366873127623544061835...,8,5000


In [13]:
if (not support_partial) & (results.ordered_associated_liquidity.sum() < abs(inputAmount)):
    print('Insufficient Liquidity')
else:
    passed_indexes = AlphaRouter.gen_one_order_selector(results.ordered_associated_liquidity, abs(inputAmount), threshold_orders)
    top_n_threshold_orders = [results.indexes[i] for i in passed_indexes]
order_subset = [orders[i] for i in top_n_threshold_orders]
top_n_threshold_orders

[1, 3, 5, 7, 9, 0]

## Exact Matching with Marginal Price Router

In [14]:
def mpr_match(inputAmount, order_subset):
    total_subset_liquidity = sum(o.y for o in order_subset)
    print('inputAmount', inputAmount)
    print('total_subset_liquidity', total_subset_liquidity)
    if inputAmount>= total_subset_liquidity:
        print(f'Partial Match - using {total_subset_liquidity}')
        inputAmount = total_subset_liquidity
        rl1 = [o.y for o in order_subset]
        rl2 = [o.dxfromdy_f(o.y) for o in order_subset]
    else:
        dy_f = lambda p: sum(o.dyfromp_f(p) for o in order_subset)
        dx_f = lambda p: sum(o.dxfromdy_f(o.dyfromp_f(p)) for o in order_subset)
        p_goal = CarbonOrderUI.goalseek(lambda p: dy_f(p)-inputAmount, Decimal('0.000000001'), Decimal('1000000000'))
        rl1 = [o.dyfromp_f(p_goal) for o in order_subset]
        rl2 = [o.dxfromdy_f(o.dyfromp_f(p_goal)) for o in order_subset]

    actions = {top_n_threshold_orders[i]:{"input":rl1[i],"output":rl2[i]} for i in range(len(top_n_threshold_orders))}
    assertAlmostEqual(inputAmount, sum(rl1), Decimal('1E-8'))
    print('total_input',sum(rl1))
    print('total_output', sum(rl2))
    print('effective_price', sum(rl2) / sum(rl1))
    return(actions)


In [15]:
mpr_match(inputAmount, order_subset)

inputAmount 5000
total_subset_liquidity 25000
total_input 5000.0000006229186112755629630626899617772067018230
total_output 11180339.888744785704593606336299168160465382833036
effective_price 2236.0679774703794690700001492603664356896345229263


{1: {'input': Decimal('1.245837222551125926125379923554413403646E-7'),
  'output': Decimal('0.00024916744450954759852855739665245820699584541663337')},
 3: {'input': Decimal('1.245837222551125926125379923554413403646E-7'),
  'output': Decimal('0.00024916744450954759852855739665245820699584541663337')},
 5: {'input': Decimal('1.245837222551125926125379923554413403646E-7'),
  'output': Decimal('0.00024916744450954759852855739665245820699584541663337')},
 7: {'input': Decimal('1.245837222551125926125379923554413403646E-7'),
  'output': Decimal('0.00024916744450954759852855739665245820699584541663337')},
 9: {'input': Decimal('1.245837222551125926125379923554413403646E-7'),
  'output': Decimal('0.00024916744450954759852855739665245820699584541663337')},
 0: {'input': Decimal('5000'),
  'output': Decimal('11180339.887498948482045868343656381177203091798057')}}